In [60]:
"""
Day 2 of kaggle's SQL Scavenger
"""

from google.cloud import bigquery
import pandas as pd

def print_schema(client, project, dataset, table):
    table_ref = client.dataset(dataset, project=project).table(table)
    table_schema = [s for s in client.get_table(table_ref).schema]
    cols = [s for s in table_schema]
    print(cols)
    
def get_rows_as_df(client, project, dataset, table, max_rows=5):
    """
    @returns a DataFrame with max_rows rows from project.dataset.table using bigquery
    """
    table_ref = client.dataset(dataset, project=project).table(table)
    table_schema = [s for s in client.get_table(table_ref).schema]
    cols = [s.name for s in table_schema]
    results = client.list_rows(table_ref, max_results=max_rows, selected_fields=table_schema)
    return pd.DataFrame(
        data=[x.values() for x in results], 
        columns=cols)

def get_query_results_as_df(client, query):
    query_job = client.query(query)
    rows = list(query_job.result(timeout=30))
    if len(rows) == 0:
        print("query returned no rows")
        return pd.DataFrame()
    return pd.DataFrame(
        data=[list(x.values()) for x in rows], 
        columns=list(rows[0].keys()))

client = bigquery.Client()
get_rows_as_df(client, 'bigquery-public-data', 'hacker_news', 'comments')
print_schema(client, 'bigquery-public-data', 'hacker_news', 'comments')

[SchemaField('id', 'integer', 'NULLABLE', 'Unique comment ID', ()), SchemaField('by', 'string', 'NULLABLE', 'Username of commenter', ()), SchemaField('author', 'string', 'NULLABLE', 'Username of author', ()), SchemaField('time', 'integer', 'NULLABLE', 'Unix time', ()), SchemaField('time_ts', 'timestamp', 'NULLABLE', 'Human readable time in UTC (format: YYYY-MM-DD hh:mm:ss)', ()), SchemaField('text', 'string', 'NULLABLE', 'Comment text', ()), SchemaField('parent', 'integer', 'NULLABLE', 'Parent comment ID', ()), SchemaField('deleted', 'boolean', 'NULLABLE', 'Is deleted?', ()), SchemaField('dead', 'boolean', 'NULLABLE', 'Is dead?', ()), SchemaField('ranking', 'integer', 'NULLABLE', 'Comment ranking', ())]


In [48]:
query = """
        SELECT parent, COUNT(id)
        FROM `bigquery-public-data.hacker_news.comments`
        GROUP BY parent
        HAVING COUNT(id) > 10
        """
popular_parents= list(get_query_results_as_df(client, query)['parent'])
print(popular_parents[0:10])

[5371725, 7833250, 3652041, 2403696, 8857265, 9460555, 9531224, 6816104, 547859, 3258254]


In [58]:
query = """
        SELECT *
        FROM `bigquery-public-data.hacker_news.comments`
        WHERE id > 216230"""

get_query_results_as_df(client, q)

query returned no rows


""


IndexError: list index out of range